# Práctica de redes neuronales

### Inteligencia Artificial
### Grado en Ingeniería Informática - Ingeniería del Software
### Universidad de Sevilla

[Keras](https://keras.io/) es una biblioteca de Python que proporciona una interfaz amigable, modular y extensible para experimentar con redes neuronales. Actualmente Keras se desarrolla como parte de [TensorFlow](https://www.tensorflow.org/), la plataforma para aprendizaje automático de Google. Esto significa que para poder usar Keras se debe instalar el paquete `tensorflow` de Python. En esta práctica también se hará uso de los paquetes `numpy`, `pandas` y `sklearn`, que se deberán tener instalados.

Trabajar con redes neuronales implica el manejo de números [pseudoaleatorios](https://es.wikipedia.org/wiki/N%C3%BAmero_pseudoaleatorio). Para que este documento sea reproducible es necesario, por tanto, establecer una semilla inicial para el generador de esos números.

In [1]:
from tensorflow import random as tensorflow_random

tensorflow_random.set_seed(394867)

Importamos en primer lugar los paquetes que nos permitirán preprocesar los datos y dividirlos en subconjuntos de entrenamiento y prueba. También establecemos una semilla inicial para que estas operaciones sean reproducibles y fijamos el tamaño máximo que debe tener un array para que se muestre completo.

In [2]:
import numpy
import pandas
from sklearn import model_selection

numpy.random.seed(43958734)
numpy.set_printoptions(threshold=10)

Finalmente importamos el paquete `keras`.

In [3]:
from tensorflow import keras

## Redes neuronales para tareas de regresión

El conjunto de datos [Gas Turbine CO and NOx Emission](http://archive.ics.uci.edu/ml/datasets/Gas+Turbine+CO+and+NOx+Emission+Data+Set) del repositorio [UCI](http://archive.ics.uci.edu/ml/index.html) contiene las siguientes medidas, agregadas por hora y tomadas desde el 1 de enero de 2011 hasta el 31 de diciembre de 2015, de una turbina de gas localizada en la región noroccidental de Turquía:

* AT: temperatura ambiental, en grados centígrados.
* AP: presión ambiental, en milibares.
* AH: humedad ambiental, en porcentaje.
* AFDP: diferencia de presión en el filtro de aire, en milibares.
* GTEP: presión de escape de la turbina de gas, en milibares.
* TIT: temperatura de entrada de la turbina, en grados centígrados.
* TAT: temperatura de salida de la turbina, en grados centígrados.
* CDP: presión de descarga del compresor, en milibares.
* TEY: rendimiento energético de la turbina, en megavatios por hora.
* CO: emisiones de monóxido de carbono, en miligramos por metro cúbico.
* NOX: emisiones de óxidos de nitrógeno, en miligramos por metro cúbico.

Los datos se encuentran en el fichero `gas_turbine.csv`.

In [ ]:
gas_turbine = pandas.read_csv("gas_turbine.csv")
gas_turbine.head()

Se plantea el problema de predecir el rendimiento energético de la turbina a partir de las medidas ambientales (AT, AP y AH) y de las medidas de los sensores de la turbina (AFDP, GTEP, TIT, TAT, CDP). Obsérvese que se trata de un problema de regresión, puesto que la variable objetivo es una variable continua.

En primer lugar, seleccionamos por un lado las variables predictoras y por otro lado la variable respuesta. En ambos casos transformamos los resultados a un array de NumPy, ya que ese tipo de dato es compatible con Keras mientras que los marcos de datos de Pandas no lo son.

In [ ]:
atributos = gas_turbine.loc[:, 'AT':'CDP']
atributos = atributos.to_numpy()
print(atributos)

In [ ]:
objetivo = gas_turbine['TEY']
objetivo = objetivo.to_numpy()
print(objetivo)

A continuación construimos los subconjuntos de entrenamiento y prueba para la construcción y evaluación de redes neuronales mediante aprendizaje supervisado.

In [ ]:
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
    atributos, objetivo, test_size=.33)

Estamos ya en condiciones de construir una red neuronal que nos permita abordar el problema.

Keras admite dos aproximaciones a la hora de construir una red neuronal: el modelo funcional, más flexible, y el modelo secuencial, más simple y el cual será el que usaremos en esta práctica.

En el modelo secuencial se construye una red neuronal capa a capa, comenzando por la capa de entrada y continuando con el resto de capas hasta la última, que será la capa de salida.

La capa de entrada se construye como instancia de la clase `Input`, indicando la forma de la entrada, que puede ser un array con cualquier número de dimensiones. En nuestro caso la entrada será un array unidimensional con los valores de los atributos, pero podría ser por ejemplo una imagen bidimensional, etcétera.

Para construir una red neuronal con alimentación hacia adelante, en el que las capas están totalmente conectadas, hay que añadir instancias de la clase `layers.Dense`, indicando la cantidad de neuronas y la función de activación (por defecto, la identidad).

Por ejemplo, podemos construir una red neuronal con ocho neuronas de entrada, una por cada atributo, y una neurona de salida, que proporcione el rendimiento energético predicho, de la siguiente manera:

In [ ]:
red_turbina = keras.Sequential()
red_turbina.add(keras.Input(shape=(8,)))
red_turbina.add(keras.layers.Dense(1))

El método `summary` nos muestra la estructura de la red, indicando para cada capa la forma de su salida y cuántos parámetros posee. La capa de entrada no se muestra, ya que no posee parámetros. La primera dimensión en la forma de la salida de cada capa indica el tamaño de los lotes (de los que hablaremos posteriormente) y si su valor es `None` quiere decir que se determinará posteriormente.

In [ ]:
red_turbina.summary()

Se observa cómo la red neuronal posee una única capa (aparte de la de entrada) que proporciona como salida un array bidimensional None x 1 (es decir, para cada ejemplo proporciona un valor) y 9 parámetros (el peso de la conexión de cada entrada con la neurona de la capa más el sesgo de la neurona) entrenables (la red los aprenderá mediante el algoritmo de entrenamiento).

Los pesos y sesgos de la red se guardan en el atributo `weights` (son los arrays asociados al argumento numpy en la estructura de datos guardada en ese atributo).

In [ ]:
red_turbina.weights

Si le pedimos a la red que prediga el rendimiento energético de los cinco primeros ejemplos del conjunto de datos, se obtendrán valores muy distintos de los correctos mostrados anteriormente, ya que todavía no se ha entrenado la red.

In [ ]:
red_turbina.predict(gas_turbine.iloc[0:5, 0:8].to_numpy())

Para entrenar una red neuronal hay que compilarla primero, estableciendo el algoritmo de aprendizaje (*optimizer*) y la función de pérdida (*loss*) a minimizar. Lo más cercano a lo explicado en clase es usar el error cuadrático medio (Keras no tiene implementada la suma de los errores cuadráticos, pero la minimización de ambas funciones es equivalente) y el algoritmo del descenso estocástico por el gradiente (*stochastic gradient descent*, SGD).

El descenso estocástico por el gradiente es el algoritmo de retropropagación, pero en lugar de actualizar los pesos tras procesar todos los ejemplos de entrenamiento lo hace tras procesar un subconjunto de ellos cada vez. Cada uno de estos subconjuntos se llama un lote (*batch*). Los lotes se construyen repartiendo aleatoriamente los ejemplos de entrenamiento y cuando se han considerado todos los lotes (y, en consecuencia, todos los ejemplos) se dice que ha transcurrido una época (*epoch*).

In [ ]:
red_turbina.compile(optimizer='SGD', loss='mean_squared_error')

Por defecto se usa un factor de aprendizaje igual a 0.01. Solo falta, entonces, proporcionar los ejemplos de entrenamiento junto con la salida esperada para cada uno de ellos, el tamaño de los lotes y el número de épocas a entrenar.

In [ ]:
red_turbina.fit(atributos_entrenamiento, objetivo_entrenamiento,
                batch_size=256, epochs=10)

El entrenamiento ha fallado, ya que se han producido desbordamientos numéricos al calcular la función de pérdida (nan quiere decir *not a number*). Los motivos de que se produzca esta circunstancia pueden ser múltiples y, a veces, difíciles de determinar. En este caso el problema se encuentra en que proporcionamos los valores en bruto de los atributos, cuando lo recomendable es que los valores que reciban como entrada las neuronas sean cercanos a cero.

Es necesario, pues, introducir tras la capa de entrada una capa de normalización como por ejemplo la disponible en Keras para tipificar las variables (a cada variable se le resta su media y se divide por su desviación típica). Los parámetros de esta capa de normalización deben ajustarse a partir únicamente de los datos de entrenamiento, lo que se hace mediante el método `adapt`.

In [4]:
normalizador = keras.layers.experimental.preprocessing.Normalization()
normalizador.adapt(atributos_entrenamiento)

NameError: name 'keras' is not defined

Podemos comprobar como todos los atributos de entrenamiento tienen media y varianza aproximadamente 0 y 1, respectivamente, al ser normalizados.

In [5]:
print(numpy.mean(normalizador(atributos_entrenamiento), axis=0))
print(numpy.var(normalizador(atributos_entrenamiento), axis=0))

NameError: name 'normalizador' is not defined

Al introducir en la red esa capa de normalización justo tras la capa de entrada, todos los datos que se proporcionen como entrada a la red se tipificarán usando las medias y desviaciones típicas calculadas.

In [16]:
red_turbina = keras.Sequential()
red_turbina.add(keras.Input(shape=(8,)))
red_turbina.add(normalizador)
red_turbina.add(keras.layers.Dense(1))

Se puede comprobar que los parámetros de esa capa de normalización no son entrenables, es decir, no se modificarán al aplicar a la red el algoritmo de descenso estocástico por el gradiente (la capa de normalización tiene 17 parámetros en total porque guarda la media y la desviación típica de cada uno de los ocho atributos y, además, la cantidad total de ejemplos a partir de los cuales se han calculado).

In [17]:
red_turbina.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 8)                 17        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 26
Trainable params: 9
Non-trainable params: 17
_________________________________________________________________


Como se ha vuelto a construir la red desde cero, también hay que volverla a compilar.

In [18]:
red_turbina.compile(optimizer='SGD', loss='mean_squared_error')

Ahora la red ya entrena sin problema.

In [19]:
red_turbina.fit(atributos_entrenamiento, objetivo_entrenamiento,
               batch_size=256, epochs=10)

Epoch 1/10
97/97 [==============================] - 0s 1ms/step - loss: 8988.8457
Epoch 2/10
97/97 [==============================] - 0s 1ms/step - loss: 179.7722
Epoch 3/10
97/97 [==============================] - 0s 994us/step - loss: 5.1726
Epoch 4/10
97/97 [==============================] - 0s 862us/step - loss: 1.5482
Epoch 5/10
97/97 [==============================] - 0s 964us/step - loss: 1.4378
Epoch 6/10
97/97 [==============================] - 0s 911us/step - loss: 1.4222
Epoch 7/10
97/97 [==============================] - 0s 1ms/step - loss: 1.3627
Epoch 8/10
97/97 [==============================] - 0s 937us/step - loss: 1.3724
Epoch 9/10
97/97 [==============================] - 0s 932us/step - loss: 1.3722
Epoch 10/10
97/97 [==============================] - 0s 951us/step - loss: 1.3224


Para comprobar el comportamiento de la red sobre los datos de prueba basta usar el método `evaluate`.

In [20]:
red_turbina.evaluate(atributos_prueba, objetivo_prueba)

379/379 [==============================] - 1s 1ms/step - loss: 1.2981


1.2981092929840088

Una forma de tratar de mejorar el rendimiento de la red neuronal es aumentar el número de épocas de entrenamiento. Esto, aparte del obvio coste en tiempo que supone, no garantiza que el algoritmo de entrenamiento no se quede atascado en un mínimo local de la función de pérdida, sin llegar a aproximarse nunca al mínimo global.

Otra vía para conseguir una red neuronal con mejor rendimiento es modificar la estructura de la misma. Por ejemplo, en el caso que nos ocupa, podemos probar a incluir una capa oculta de la que se ha elegido, de forma arbitraria, que tenga 10 neuronas.

In [21]:
red_turbina = keras.Sequential()
red_turbina.add(keras.Input(shape=(8,)))
red_turbina.add(normalizador)
red_turbina.add(keras.layers.Dense(10))
red_turbina.add(keras.layers.Dense(1))

Esta red tiene 101 parámetros entrenables, que provienen de los 8x10 pesos más 10 sesgos de las neuronas de la capa oculta más los 10 pesos y el sesgo de la neurona de la capa de salida. También tiene 17 parámetros no entrenables, que provienen de la capa de normalización.

In [22]:
red_turbina.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 8)                 17        
_________________________________________________________________
dense_2 (Dense)              (None, 10)                90        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 118
Trainable params: 101
Non-trainable params: 17
_________________________________________________________________


Al compilar la red se puede indicar también que, aparte de la función de pérdida, se calculan otras métricas adecuadas. Por ejemplo, el error absoluto medio (es decir, la media de las diferencias en valor absoluto entre los valores predichos y los valores correctos) es una métrica adecuada para una tarea de regresión.

In [23]:
red_turbina.compile(optimizer='SGD', loss='mean_squared_error',
                    metrics=['mean_absolute_error'])

In [24]:
red_turbina.fit(atributos_entrenamiento, objetivo_entrenamiento,
               batch_size=256, epochs=10)

Epoch 1/10
97/97 [==============================] - 0s 1ms/step - loss: nan - mean_absolute_error: nan        
Epoch 2/10
97/97 [==============================] - 0s 1ms/step - loss: nan - mean_absolute_error: nan
Epoch 3/10
97/97 [==============================] - 0s 1ms/step - loss: nan - mean_absolute_error: nan
Epoch 4/10
97/97 [==============================] - 0s 1ms/step - loss: nan - mean_absolute_error: nan
Epoch 5/10
97/97 [==============================] - 0s 2ms/step - loss: nan - mean_absolute_error: nan
Epoch 6/10
97/97 [==============================] - 0s 1ms/step - loss: nan - mean_absolute_error: nan
Epoch 7/10
97/97 [==============================] - 0s 986us/step - loss: nan - mean_absolute_error: nan
Epoch 8/10
97/97 [==============================] - 0s 1ms/step - loss: nan - mean_absolute_error: nan
Epoch 9/10
97/97 [==============================] - 0s 972us/step - loss: nan - mean_absolute_error: nan
Epoch 10/10
97/97 [==============================] - 0s 1ms/s

De nuevo se han producido desbordamientos en el cálculo de la función de pérdida. En este caso es debido a que las neuronas de la capa oculta tienen como función de activación a la función identidad (la función de activación por defecto). Por lo tanto, aunque ellas sí reciben las entradas normalizadas, los valores que devuelven, y que son las que recibe la neurona de la capa de salida, pueden alejarse bastante del valor cero. Si establecemos como función de activación de las neuronas de la capa oculta a la función sigmoide, esta normalizará al intervalo (0, 1) los valores que devuelven esas neuronas y la red se podrá entrenar sin problema.

In [25]:
red_turbina = keras.Sequential()
red_turbina.add(keras.Input(shape=(8,)))
red_turbina.add(normalizador)
red_turbina.add(keras.layers.Dense(10, activation='sigmoid'))
red_turbina.add(keras.layers.Dense(1))

Al compilar la red se pueden cambiar los parámetros de los distintos argumentos proporcionando, en lugar del nombre, una instancia de la clase que lo implementa. Por ejemplo, para establecer el factor de aprendizaje, en lugar de proporcionar el nombre `'SGD'` para indicar descenso estocástico por el gradiente como optimizador hay que proporcionar una instancia de la clase `SGD` que lo implementa.

In [26]:
red_turbina.compile(optimizer=keras.optimizers.SGD(learning_rate=0.02), loss='mean_squared_error',
                    metrics=['mean_absolute_error'])

In [27]:
red_turbina.fit(atributos_entrenamiento, objetivo_entrenamiento,
               batch_size=256, epochs=10)

Epoch 1/10
97/97 [==============================] - 0s 1ms/step - loss: 1986.0117 - mean_absolute_error: 22.1891
Epoch 2/10
97/97 [==============================] - 0s 1ms/step - loss: 4.9152 - mean_absolute_error: 1.4067
Epoch 3/10
97/97 [==============================] - 0s 1ms/step - loss: 2.2173 - mean_absolute_error: 1.0041
Epoch 4/10
97/97 [==============================] - 0s 1ms/step - loss: 1.5979 - mean_absolute_error: 0.8889
Epoch 5/10
97/97 [==============================] - 0s 1ms/step - loss: 1.3663 - mean_absolute_error: 0.8433
Epoch 6/10
97/97 [==============================] - 0s 1ms/step - loss: 1.1798 - mean_absolute_error: 0.7952
Epoch 7/10
97/97 [==============================] - 0s 2ms/step - loss: 1.0375 - mean_absolute_error: 0.7627
Epoch 8/10
97/97 [==============================] - 0s 1ms/step - loss: 0.9815 - mean_absolute_error: 0.7439
Epoch 9/10
97/97 [==============================] - 0s 2ms/step - loss: 0.9552 - mean_absolute_error: 0.7395
Epoch 10/10
97/

In [28]:
red_turbina.evaluate(atributos_prueba, objetivo_prueba)

379/379 [==============================] - 1s 1ms/step - loss: 0.8529 - mean_absolute_error: 0.7061


[0.852899432182312, 0.7061452269554138]

Se obtiene un error sobre el conjunto de prueba menor que el obtenido con la red sin capa oculta.

### **Ejercicio 1**

Se plantea el problema de predecir **a la vez** el rendimiento energético de la turbina (TEY), las emisiones de monóxido de carbono (CO) y las emisiones de óxidos de nitrógeno (NOX) a partir de las medidas ambientales (AT, AP y AH) y de las medidas de los sensores de la turbina (AFDP, GTEP, TIT, TAT, CDP).

Experimentar con diferentes arquitecturas de redes neuronales, que necesariamente deberán tener 3 neuronas en la capa de salida, y con diferentes configuraciones de entrenamiento para abordar el problema.

El objetivo último es construir una red neuronal con un error absoluto medio sobre el conjunto de prueba de alrededor de 1.5.

In [29]:
objetivos = gas_turbine.loc[:, 'TEY':'NOX']
objetivos = objetivos.to_numpy()
print(objetivos)

[[134.67      0.32663  81.952  ]
 [134.67      0.44784  82.377  ]
 [135.1       0.45144  83.776  ]
 ...
 [107.81     11.414    96.147  ]
 [131.41      3.3134   64.738  ]
 [125.41     11.981   109.24   ]]


In [30]:
(atributos_entrenamiento, atributos_prueba,
 objetivos_entrenamiento, objetivos_prueba) = model_selection.train_test_split(
    atributos, objetivos, test_size=.33)

In [31]:
red_turbina = keras.Sequential()
red_turbina.add(keras.Input(shape=(8,)))
red_turbina.add(normalizador)
red_turbina.add(keras.layers.Dense(100, activation='sigmoid'))
red_turbina.add(keras.layers.Dense(100, activation='sigmoid'))
red_turbina.add(keras.layers.Dense(3))

In [32]:
red_turbina.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 8)                 17        
_________________________________________________________________
dense_6 (Dense)              (None, 100)               900       
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 303       
Total params: 11,320
Trainable params: 11,303
Non-trainable params: 17
_________________________________________________________________


In [33]:
red_turbina.compile(optimizer=keras.optimizers.SGD(learning_rate=0.05), loss='mean_squared_error',
                    metrics=['mean_absolute_error'])

In [34]:
red_turbina.fit(atributos_entrenamiento, objetivos_entrenamiento,
                batch_size=256, epochs=300, verbose=0)

In [35]:
red_turbina.evaluate(atributos_prueba, objetivos_prueba)

379/379 [==============================] - 0s 843us/step - loss: 8.1771 - mean_absolute_error: 1.5575


[8.177083969116211, 1.5575082302093506]

## Redes neuronales para tareas de clasificación

*Iris* es un conjunto de datos multivariante que se ha estudiado exhaustivamente y se ha convertido en un conjunto de datos de referencia a la hora de analizar el comportamiento de los distintos algoritmos de aprendizaje automático.

*Iris* recopila cuatro medidas (longitud y anchura de sépalo y pétalo) de 50 flores de cada una de las siguientes tres especies de lirios: *Iris setosa*, *Iris virginica* e *Iris versicolor*.

Los datos se encuentran en el fichero `iris.csv`.

In [36]:
iris = pandas.read_csv('iris.csv', header=None,
                       names=['Longitud sépalo', 'Anchura sépalo',
                              'Longitud pétalo', 'Anchura pétalo',
                              'Especie'])
iris.head()

,Longitud sépalo,Anchura sépalo,Longitud pétalo,Anchura pétalo,Especie
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Se plantea el problema de predecir si una flor es o no de la especie *Iris setosa* a partir de las medidas de sus sépalos y pétalos. Obsérvese que se trata de un problema de clasificación, puesto que la variable objetivo es una variable categórica binaria.

En primer lugar, seleccionamos por un lado las variables predictoras y por otro lado la variable respuesta. En ambos casos transformamos los resultados a un array de NumPy. La variable respuesta la transformamos además a una variable numérica.

In [37]:
atributos = iris.loc[:, 'Longitud sépalo':'Anchura pétalo']
atributos = atributos.to_numpy()
print(atributos)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 ...
 [6.5 3.  5.2 2. ]
 [6.2 3.4 5.4 2.3]
 [5.9 3.  5.1 1.8]]


In [38]:
objetivo = iris['Especie'] == 'Iris-setosa'
objetivo = objetivo.to_numpy().astype(float)
print(objetivo)

[1. 1. 1. ... 0. 0. 0.]


A continuación construimos los subconjuntos de entrenamiento y prueba.

In [40]:
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
    atributos, objetivo, test_size=.33)

Estamos ya en condiciones de construir una red neuronal que nos permita abordar el problema. Al tratarse este de un problema de clasificación binaria es habitual considerar una única neurona de salida que devuelva un valor entre 0 y 1 representando la probabilidad de que el ejemplo de entrada pertenezca a la clase positiva.

In [41]:
normalizador = keras.layers.experimental.preprocessing.Normalization()
normalizador.adapt(atributos_entrenamiento)

In [42]:
red_lirios = keras.Sequential()
red_lirios.add(keras.Input(shape=(4,)))
red_lirios.add(normalizador)
red_lirios.add(keras.layers.Dense(1, activation='sigmoid'))

In [43]:
red_lirios.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_1 (Normalizati (None, 4)                 9         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 5         
Total params: 14
Trainable params: 5
Non-trainable params: 9
_________________________________________________________________


La función de pérdida a minimizar adecuada en este caso es la entropía cruzada binaria:
$$C(y, \hat{y}) = -y \log(\hat{y}) - (1 - y)\log(1 - \hat{y})$$
donde $y$ es la respuesta correcta e $\hat{y}$ es la respuesta proporcionada por el modelo.

También le vamos a pedir a la red que calcule su exactitud, es decir, la fracción de ejemplos clasificados correctamente. Una flor se clasificará como de la especie *Iris setosa* si la probabilidad de que sea así devuelta por la red es mayor que 0.5.

In [44]:
red_lirios.compile(optimizer='SGD', loss='binary_crossentropy',
                   metrics=['accuracy'])

In [45]:
red_lirios.fit(atributos_entrenamiento, objetivo_entrenamiento,
               batch_size=10, epochs=10)

Epoch 1/10
10/10 [==============================] - 0s 1ms/step - loss: 0.7846 - accuracy: 0.4001
Epoch 2/10
10/10 [==============================] - 0s 2ms/step - loss: 0.7330 - accuracy: 0.5435
Epoch 3/10
10/10 [==============================] - 0s 2ms/step - loss: 0.6697 - accuracy: 0.5892
Epoch 4/10
10/10 [==============================] - 0s 2ms/step - loss: 0.5781 - accuracy: 0.6627
Epoch 5/10
10/10 [==============================] - 0s 2ms/step - loss: 0.5511 - accuracy: 0.7772
Epoch 6/10
10/10 [==============================] - 0s 2ms/step - loss: 0.5084 - accuracy: 0.8833
Epoch 7/10
10/10 [==============================] - 0s 1ms/step - loss: 0.4544 - accuracy: 0.9021
Epoch 8/10
10/10 [==============================] - 0s 1ms/step - loss: 0.4483 - accuracy: 0.9680
Epoch 9/10
10/10 [==============================] - 0s 1ms/step - loss: 0.4277 - accuracy: 0.9296
Epoch 10/10
10/10 [==============================] - 0s 1ms/step - loss: 0.3797 - accuracy: 0.9725


In [46]:
red_lirios.evaluate(atributos_prueba, objetivo_prueba)

2/2 [==============================] - 0s 2ms/step - loss: 0.3627 - accuracy: 0.9800


[0.36273708939552307, 0.9800000190734863]

En este caso la red construida es capaz de identificar correctamente todas las flores del conjunto de prueba que son de la especie *Iris setosa*. Esto no es sorprendente, ya que es perfectamente conocido que las medidas de los sépalos y pétalos permiten separar esta especie de lirio de las otras dos especies. Separar estas dos últimas entre sí ya entraña un poco de mayor dificultad.

### **Ejercicio 2**

Se plantea el problema de determinar la especie de lirio a la que pertenece una flor a partir de las medidas de sus sépalos y pétalos.

Experimentar con diferentes arquitecturas de redes neuronales y con diferentes configuraciones de entrenamiento para abordar el problema.

Puesto que se trata de un problema de clasificación multiclase, es conveniente que la capa de salida tenga tantas neuronas como clases y que la función de activación para estas neuronas sea la función softmax, que dada la tupla de entradas de las neuronas las normaliza a valores en el intervalo (0, 1) representando la probabilidad de pertenencia a a cada clase:
$$\mathit{softmax}(z_{1}, \ldots, z_{n}) =
\left(\frac{\mathrm{e}^{z_{1}}}{\sum_{i = 1}^{n} \mathrm{e}^{z_{i}}}, \ldots,
\frac{\mathrm{e}^{z_{n}}}{\sum_{i = 1}^{n} \mathrm{e}^{z_{i}}}\right)$$

De esta forma, cada ejemplo se clasificará en la clase más probable.

La función de pérdida a minimizar adecuada en este caso es la entropía cruzada categórica (`'categorical_crossentropy'`)
$$C(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{i = 1}^{n} \mathbf{y}_{i}⁢ \log\hat{\mathbf{y}}_{i}$$
donde $\mathbf{y}$ es la codificación *one-hot* de la clase correcta e $\hat{\mathbf{y}}$ es el vector de probabilidades de pertenencia a cada clase proporcionado por el modelo.

Para codificar la variable objetivo mediante *one-hot* es útil la función `get_dummies` de la biblioteca Pandas.

In [47]:
objetivo = iris['Especie']
objetivo = pandas.get_dummies(objetivo)
print(objetivo)
objetivo = objetivo.to_numpy()
print(objetivo)

     Iris-setosa  Iris-versicolor  Iris-virginica
0              1                0               0
1              1                0               0
2              1                0               0
3              1                0               0
4              1                0               0
..           ...              ...             ...
145            0                0               1
146            0                0               1
147            0                0               1
148            0                0               1
149            0                0               1

[150 rows x 3 columns]
[[1 0 0]
 [1 0 0]
 [1 0 0]
 ...
 [0 0 1]
 [0 0 1]
 [0 0 1]]


El objetivo último es construir una red neuronal con una exactitud sobre el conjunto de prueba superior a 0.9.

In [48]:
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
    atributos, objetivo, test_size=.33)

In [49]:
normalizador = keras.layers.experimental.preprocessing.Normalization()
normalizador.adapt(atributos_entrenamiento)

In [50]:
red_lirios = keras.Sequential()
red_lirios.add(keras.Input(shape=(4,)))
red_lirios.add(normalizador)
red_lirios.add(keras.layers.Dense(50, activation='tanh'))
red_lirios.add(keras.layers.Dense(3, activation='softmax'))

In [51]:
red_lirios.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_2 (Normalizati (None, 4)                 9         
_________________________________________________________________
dense_10 (Dense)             (None, 50)                250       
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 153       
Total params: 412
Trainable params: 403
Non-trainable params: 9
_________________________________________________________________


In [52]:
red_lirios.compile(optimizer='SGD', loss='categorical_crossentropy',
                   metrics=['accuracy'])

In [56]:
red_lirios.fit(atributos_entrenamiento, objetivo_entrenamiento,
               batch_size=10, epochs=200, verbose=0)

In [57]:
red_lirios.evaluate(atributos_prueba, objetivo_prueba)

2/2 [==============================] - 0s 4ms/step - loss: 0.0984 - accuracy: 0.9600


[0.09842399507761002, 0.9599999785423279]